This notebooks checks the approximation quality of a distribution derived from quantile values.

In [1]:
import numpy as np
import pandas as pd
import math
import scipy.stats as stats
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from naslib.optimizers.bananas.distribution import get_quantile_levels
from naslib.optimizers.bananas.distribution import Distribution, PointwiseInterpolatedDist

In [2]:
sns.set_style("darkgrid")
rcParams['axes.titlepad'] = 20 
rcParams['font.size'] = 9

### Define functions

In [3]:
def fit_distribution(samples, num_quantiles) -> PointwiseInterpolatedDist:
    sample_size = len(samples)

    pk = get_quantile_levels(num_quantiles=num_quantiles)
    qk = []
    for p in pk:
        adj_p = min(math.ceil((sample_size + 1) * p) / sample_size, 1.0)
        qk.append(np.quantile(samples, adj_p))
    
    return PointwiseInterpolatedDist(values=(pk, np.array(qk)))

In [4]:
def kl_divergence(distribution: Distribution, other: Distribution, x) -> float:
    """Kullback-Leibler-Divergence."""
    y = [distribution.pdf(x_i) * np.log(distribution.pdf(x_i) / other.pdf(x_i)) for x_i in x]
    return np.trapz(y, x)

### Statistics

In [5]:
def _single_dist_stats(self, other, x):
    return pd.DataFrame(
        {"mean": self.mean(), "std": self.std(), "KL divergence":  kl_divergence(self, other=other, x=x)}, index=[0]
    )

In [6]:
num_quantiles = 20
sample_sizes = np.arange(50, 501, 50)

# True distribution
true_dist = stats.skewnorm(loc=0, scale=1, a=-10)
mean = true_dist.mean()
std = true_dist.std()


statistics = {}
# Fit distributions
x = np.linspace(mean - 3 * std, mean + 3 * std, 1000)
for n_sample in sample_sizes:
    group_name = f"sample size={n_sample}"
    samples = true_dist.rvs(size=n_sample, random_state=5)

    # normal distribuion
    loc, scale = stats.norm.fit(samples)
    norm_dist = stats.norm(loc=loc, scale=scale)
    statistics[(group_name, "Est. Gaussian")] = _single_dist_stats(self=norm_dist, other=true_dist, x=x)
    # point-wise distribution
    pointwise_dist = fit_distribution(samples=samples, num_quantiles=20)
    statistics[(group_name, "Est. Quantile")] = _single_dist_stats(self=pointwise_dist, other=true_dist, x=x)

statistics[("", "Actual")] = _single_dist_stats(self=true_dist, other=true_dist, x=x)

In [ ]:
df = pd.concat(statistics, axis=0).droplevel(level=2)
df

### Graphical Anylsis

In [8]:
output_dir = "./figs/distribution_quality"

In [9]:
num_quantiles = 20
sample_size = 500

In [10]:
true_dist = stats.skewnorm(loc=0, scale=1, a=-10)
true_dist.name = "Actual"

samples = true_dist.rvs(size=sample_size)
pointwise_dist = fit_distribution(samples=samples, num_quantiles=num_quantiles)
pointwise_dist.name = "Est. Quantile"

loc, scale = stats.norm.fit(samples)
norm_dist = stats.norm(loc=loc, scale=scale)
norm_dist.name = "Est. Gaussian"

#### Density Plot

In [11]:
dist_name = true_dist.name
mean = true_dist.mean()
std = true_dist.std()

In [ ]:
fig = plt.figure()

# plot true distribution
x = np.linspace(mean - 3 * std, mean + 3 * std, 1000)
plt.plot(x, true_dist.pdf(x), label=true_dist.name, alpha=0.8, color="blue")

# plot estimated distribution
densities = [pointwise_dist.pdf(x_i) for x_i in x]
plt.plot(x, densities, label=pointwise_dist.name, alpha=0.8, color="red")

plt.plot(x, norm_dist.pdf(x), label=norm_dist.name, alpha=0.8, color="green")

plt.legend(loc="upper left")
plt.xlabel("x")
plt.ylabel("pdf(x)")
plt.tight_layout()

In [28]:
# save
# fig.savefig(output_dir + f"/{title}_pdf_plot.pdf")

#### CDF Plot

In [ ]:
fig = plt.figure()

# plot true distribution
x = np.linspace(mean - 4 * std, mean + 3 * std, 100)
plt.plot(x, true_dist.cdf(x), label=true_dist.name, alpha=0.8, color="blue")
# plot estimated distribution 
cdfs = [pointwise_dist.cdf(x_i) for x_i in x]
plt.plot(x, cdfs, label=pointwise_dist.name, alpha=0.8, color="red")

plt.plot(x, norm_dist.cdf(x), label=norm_dist.name, alpha=0.8, color="green")

plt.legend(loc="upper left")
plt.xlabel("x")
plt.ylabel("cdf(x)")
plt.tight_layout()

In [16]:
# save
# fig.savefig(output_dir + f"/{title}_cdf_plot.pdf")

### Sampling Histogram Plot

In [21]:
n_samples = 1000
true_samples = true_dist.rvs(size=n_samples)
bins = np.histogram_bin_edges(true_samples, bins=100)

In [ ]:
fig = plt.figure()
alpha = 0.8
# plot true distribution
plt.hist(true_samples, bins=bins, alpha=alpha, label=true_dist.name, color="blue")

# plot estimated distribution 
plt.hist(pointwise_dist.rvs(size=n_samples), bins=bins, alpha=alpha, label=pointwise_dist.name, color="red")
plt.hist(norm_dist.rvs(size=n_samples, random_state=111), bins=bins, alpha=0.6, label=norm_dist.name, color="green")

plt.legend(loc="upper left") 
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.tight_layout()

In [14]:
# save
# fig.savefig(output_dir + f"/{title}_rvs_plot.pdf")